In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
input_size = 784
hidden_size = 500
num_classes=10
num_epochs = 5
batch_size =100
learning_rate = 0.001

In [12]:
train_dateset = torchvision.datasets.MNIST(root='data',train=True,transform=transforms.ToTensor(),download=True)

In [13]:
test_dataset = torchvision.datasets.MNIST(root='data',train=False,transform=transforms.ToTensor())

In [16]:
train_loader = torch.utils.data.DataLoader(dataset=train_dateset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [20]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        out=self.fc1(x)
        out=self.relu(out)
        out=self.fc2(out)
        return out

In [22]:
model = NeuralNet(input_size,hidden_size,num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [25]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        #forward pass
        outputs = model(images)
        loss=criterion(outputs,labels)
        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        
        if (i+1)%100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                    

Epoch [1/5], Step [100/600], Loss: 0.3384
Epoch [1/5], Step [200/600], Loss: 0.3259
Epoch [1/5], Step [300/600], Loss: 0.2604
Epoch [1/5], Step [400/600], Loss: 0.3066
Epoch [1/5], Step [500/600], Loss: 0.1621
Epoch [1/5], Step [600/600], Loss: 0.1186
Epoch [2/5], Step [100/600], Loss: 0.0610
Epoch [2/5], Step [200/600], Loss: 0.1641
Epoch [2/5], Step [300/600], Loss: 0.1507
Epoch [2/5], Step [400/600], Loss: 0.1081
Epoch [2/5], Step [500/600], Loss: 0.0958
Epoch [2/5], Step [600/600], Loss: 0.0677
Epoch [3/5], Step [100/600], Loss: 0.0754
Epoch [3/5], Step [200/600], Loss: 0.0391
Epoch [3/5], Step [300/600], Loss: 0.0740
Epoch [3/5], Step [400/600], Loss: 0.0908
Epoch [3/5], Step [500/600], Loss: 0.0608
Epoch [3/5], Step [600/600], Loss: 0.0427
Epoch [4/5], Step [100/600], Loss: 0.0428
Epoch [4/5], Step [200/600], Loss: 0.0522
Epoch [4/5], Step [300/600], Loss: 0.0185
Epoch [4/5], Step [400/600], Loss: 0.0271
Epoch [4/5], Step [500/600], Loss: 0.1005
Epoch [4/5], Step [600/600], Loss:

In [33]:
with torch.no_grad():
    correct=0
    total=0
    for images, label in test_loader:
        images= images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        output=model(images)
        _,predicated = torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicated==labels).sum().item()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
torch.save(model.state_dict(),'model.ckpt')

Accuracy of the network on the 10000 test images: 9.78 %
